In [1]:
import datetime
import json
import re
import sys
import time

from bs4 import BeautifulSoup
import requests

In [2]:
url = "http://www.isea-archives.org/isea2019-presentations/"
r = requests.get(url)

In [3]:
soup = BeautifulSoup(r.text, "html.parser")
soup

<!DOCTYPE html>

<!--[if IE 7]>
<html class="ie ie7" lang="en">
<![endif]-->
<!--[if IE 8]>
<html class="ie ie8" lang="en">
<![endif]-->
<!--[if !(IE 7) & !(IE 8)]><!-->
<html lang="en">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<title>[ISEA2019] Presentations | ISEA Symposium Archives</title>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="http://www.isea-archives.org/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
<script src="http://www.isea-archives.org/wp-content/themes/twentytwelve/js/html5.js" type="text/javascript"></script>
<![endif]-->
<link href="//fonts.googleapis.com" rel="dns-prefetch"/>
<link href="//s.w.org" rel="dns-prefetch"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="http://www.isea-archives.org/feed/" rel="alternate" title="ISEA Symposium Archives » Feed" type="application/rss+xml"/>
<link href="http://www.isea-archives.org/comments/feed/" rel="altern

In [4]:
names = []
urls = []
link = []
for i in soup.findAll('a'):
    link.append(i)
    
for i in link:
    name = i.getText()
    url = i.attrs['href']
    names.append(name)
    urls.append(url)

In [5]:
names = names[39:160]
urls = urls[39:160]

In [6]:
url = urls[0]
r = requests.get(url)

In [7]:
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
soup

<!DOCTYPE html>

<!--[if IE 7]>
<html class="ie ie7" lang="en">
<![endif]-->
<!--[if IE 8]>
<html class="ie ie8" lang="en">
<![endif]-->
<!--[if !(IE 7) & !(IE 8)]><!-->
<html lang="en">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<title>[ISEA2019] Paper: Claudia Arozqueta — In the Spotlight: Searchlights, Art, Surveillance and Spectacle | ISEA Symposium Archives</title>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="http://www.isea-archives.org/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
<script src="http://www.isea-archives.org/wp-content/themes/twentytwelve/js/html5.js" type="text/javascript"></script>
<![endif]-->
<link href="//fonts.googleapis.com" rel="dns-prefetch"/>
<link href="//s.w.org" rel="dns-prefetch"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="http://www.isea-archives.org/feed/" rel="alternate" title="ISEA Symposium Archives » Feed" type="application/rs

In [8]:
soup.find('h1',{"entry-title"}).getText().split('Paper: ')[1].split(' — ')[0]

'Claudia Arozqueta'

In [9]:
for i in soup.find('div',{"entry-content"}).findAll('p'):
    print(i)
    print('--')

<p><strong>Abstract</strong></p>
--
<p>Keywords: <em>Searchlights, Artificial Light, Surveillance, Installation Art, Memorial Art, War, Spectacle, Media Art History.</em></p>
--
<p>Searchlights have been used historically for artistic, military, commemoration and promotion purposes. This essay begins with an outline of the historical uses of searchlights in international fairs and its deployment in political and commemorative events on the first half of the twentieth century. The second part tracks and analyzes the use of searchlights by contemporary artists who have explored the use of this medium to create spatial, personal and community results. Some artists draw lines in the sky exploring sculptural possibilities; not a few continue the traditional uses of searchlights for national celebrations; while others manage to reveal in their installations the paradoxical nature of reflectors that oscillate between being a tool of vigilance and spectacle.</p>
--
<p><a href="http://www.isea-

In [10]:
keyword = []
abstract = []
for idx, i in enumerate(soup.find('div',{"entry-content"}).findAll('p')):
    if idx == 1:
        keyword.extend(i.getText().replace('Keywords: ','').replace('\xa0',' ').split(', '))
    elif idx == 2:
        abstract.append(i.getText().replace('\xa0',' '))

In [11]:
abstract

['Searchlights have been used historically for artistic, military, commemoration and promotion purposes. This essay begins with an outline of the historical uses of searchlights in international fairs and its deployment in political and commemorative events on the first half of the twentieth century. The second part tracks and analyzes the use of searchlights by contemporary artists who have explored the use of this medium to create spatial, personal and community results. Some artists draw lines in the sky exploring sculptural possibilities; not a few continue the traditional uses of searchlights for national celebrations; while others manage to reveal in their installations the paradoxical nature of reflectors that oscillate between being a tool of vigilance and spectacle.']

In [12]:
def get_list(url):    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    keyword = []
    abstract = []
    author = soup.find('h1',{"entry-title"}).getText().split('Paper: ')[1].split(' — ')[0]
    papername = soup.find('h1',{"entry-title"}).getText().split('Paper: ')[1].split(' — ')[1]
    for idx, i in enumerate(soup.find('div',{"entry-content"}).findAll('p')):
        if idx == 1:
            keyword.append(i.getText().replace('Keywords: ','').replace('\xa0',' ').split(', '))
        elif idx == 2:
            abstract.append(i.getText().replace('\xa0',' '))
    for a,b in zip(keyword, abstract):
        document ={}
        document['keyword']=a
        document['abstract']=b
    document['author'] = author
    document['papername'] = papername
    
    return document

In [13]:
from time import sleep
final_db = {}
for idx, i in enumerate(urls):
    try:
        doc = get_list(i)
        final_db[str(idx)] = doc
        sleep(0.5)
    except:
        continue

In [14]:
len(final_db)

103

In [15]:
import json
with open('result.json', 'w') as fp:
    json.dump(final_db, fp)

In [16]:
final_db['0']

{'keyword': ['Searchlights',
  'Artificial Light',
  'Surveillance',
  'Installation Art',
  'Memorial Art',
  'War',
  'Spectacle',
  'Media Art History.'],
 'abstract': 'Searchlights have been used historically for artistic, military, commemoration and promotion purposes. This essay begins with an outline of the historical uses of searchlights in international fairs and its deployment in political and commemorative events on the first half of the twentieth century. The second part tracks and analyzes the use of searchlights by contemporary artists who have explored the use of this medium to create spatial, personal and community results. Some artists draw lines in the sky exploring sculptural possibilities; not a few continue the traditional uses of searchlights for national celebrations; while others manage to reveal in their installations the paradoxical nature of reflectors that oscillate between being a tool of vigilance and spectacle.',
 'author': 'Claudia Arozqueta',
 'papernam